In [1]:
import chromadb
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

from langchain_openai import ChatOpenAI
from langchain_text_splitters import TokenTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [2]:
llm = ChatOpenAI(temperature=0.0, model="gpt-4o", api_key= "dummy_val")

In [3]:
def get_chunks(path):
    with open(path, 'r') as file:
        content = file.read()
    text_splitter = TokenTextSplitter(chunk_size=1100, chunk_overlap=100)
    texts = text_splitter.split_text(content)
    return content, texts

def add_chunks(texts, collection):
    add_count = 0
    for text in texts:
        collection.add(documents=[text], ids=f"chunk_{add_count}")
        add_count += 1

def chroma_retrieval(query, collection, num_results=5):
    results = collection.query(query_texts=[query], n_results=num_results)
    return results

def chroma_rag(query, collection):
    retrieved_docs = chroma_retrieval(query, collection)["documents"][0]
    response = rag_chain.invoke({"retrieved_docs": retrieved_docs, "query": query})
    return retrieved_docs, response

In [4]:
main_path = "/home/aswath/Projects/deep_learning/backup_brain/test_2/"
input_path = main_path + "input/outsiders.txt"

content, texts = get_chunks(input_path)

In [21]:
client_path = main_path + "/notebook/chromadb"
chroma_client = chromadb.PersistentClient(path=client_path)
collection = chroma_client.get_or_create_collection(name="outsiders_collection")
# add_chunks(texts, collection)

In [28]:
rag_prompt_template = """
Generate a response that responds to the user's question, summarizing all information in the input data tables, and incorporating any relevant general knowledge.

Do not include information where the supporting evidence for it is not provided.

Context: {retrieved_docs}

User Question: {query}

"""

In [29]:
rag_prompt = ChatPromptTemplate.from_template(rag_prompt_template)
rag_chain = rag_prompt | llm | StrOutputParser()

In [30]:
docs, response = chroma_rag("Give me the best CEO's and elaborate reasons behind it ?", collection)

In [31]:
print(response)

The context provided highlights several CEOs who are considered exceptional due to their unique management styles and strategic decisions. Here are some of the standout CEOs and the reasons behind their success:

1. **Warren Buffett (Berkshire Hathaway)**:
   - **Capital Allocation**: Buffett is renowned for his exceptional capital allocation skills. He never paid dividends or repurchased significant amounts of stock, instead focusing on investing in publicly traded stocks and acquiring private companies. His approach allowed Berkshire to achieve remarkable returns.
   - **Concentration and Long Holding Periods**: Buffett's investment strategy involves a high degree of concentration and extremely long holding periods, which have contributed to Berkshire's success.
   - **Decisive Action**: Buffett's ability to make quick, informed decisions, such as exiting low-return businesses, has been crucial to his success.

2. **Tom Murphy (Capital Cities Broadcasting)**:
   - **Focused Strategy*

In [26]:
for i in docs:
    print(i)
    print("\n\n" + "-"*100 + "\n\n")

asm, and it invariably led to a laser-like focus on a few select variables that shaped each firm’s strategy, usually in entirely different directions from those of industry peers. For Henry Singleton in the 1970s and 1980s, it was stock buybacks; for John Malone, it was the relentless pursuit of cable subscribers; for Bill Anders, it was divesting noncore businesses; for Warren Buffett, it was the generation and deployment of insurance float.

At the core of their shared worldview was the belief that the primary goal for any CEO was to optimize long-term value per share, not organizational growth. This may seem like an obvious objective; however, in American business, there is a deeply ingrained urge to get bigger. Larger companies get more attention in the press; the executives of those companies tend to earn higher salaries and are more likely to be asked to join prestigious boards and clubs. As a result, it is very rare to see a company proactively shrink itself. And yet virtually a